In [2]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Définition du HEADERS pour que les requetes ne soit pas bloquées
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'}

# Démarrage du chronomètre pour calculer la durée d'exécution du script
start_time = time.time()

# Définition de la variable pour intervalle de progression
pourcent = 9

# Définition de la variable du nombre d'id dans l'url de JPbox-office
id_in = 1
id_out = 5000
id = id_in

# Définition de la fonction pour scraper les données de JPbox-office
def jpbox(k):
    global pourcent, id
    id += 1
    # Initialisation des variables à None
    annee = titre = budget = premiere_semaine = None
    try:

        # Print du % avec une intervalle de 24 tests
        pourcent += 1
        if pourcent == 10:
            print('Execution :', round(((id-id_in)/(id_out-id_in))*100, 2), '%')
            pourcent = 0


        # URL de la fiche de film
        url = f'https://www.jpbox-office.com/fichfilm.php?id={k}'
        # Envoyer une requête HTTP GET à l'URL
        response = requests.get(url, headers=HEADERS)
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if response.status_code != 200:
            return k, "0", "0", "0", "0"
        if response.url != url:
            return k, "0", "0", "0", "0"
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Récupération du titre
        if soup.find('h2'):
            titre = soup.find('h2')
        else:
            titre = soup.find('h1')


        # Récupération de l'année
        if soup.find('h3'):
            text = soup.find('h3').get_text(separator=" ", strip=True)
            annee = text.split('/')[0].strip()

        # Récupération du Budget
        table = soup.find_all('table', class_=lambda x: x and 'tablesmall' in x)
        for i in table:
            if i:
                table_td = i.find_all('td')
                for j in table_td:
                    if j.text == 'Budget':
                        budget = j.find_next('td')


        # URL de la fiche de film
        url = f'https://www.jpbox-office.com/fichfilm.php?id={k}&view=2'
        # Envoyer une requête HTTP GET à l'URL
        response = requests.get(url, headers=HEADERS)
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if response.status_code != 200:
            return k, "0", "0", "0", "0"
        if response.url != url:
            return k, "0", "0", "0", "0"
        # Analyser le contenu HTML de la page avec Beautiful Soup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Récupération du BO de la première semaine france
        table = soup.find_all('table', class_=lambda x: x and 'tablesmall' in x)
        for i in table:
            if i:
                table_td = i.find_all('td')
                for j in table_td:
                    if j.text == 'Première semaine':
                        premiere_semaine = j.find_next('td')
 
    
    # Retourne des "0" en cas d'exception (Pas de box office par exemple)
    except requests.exceptions.RequestException:
        return k, "0", "0", "0", "0"
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    return (
        k,
        annee if annee else "0",
        titre.text.strip() if titre else "0",
        budget.text.strip() if budget else "0",
        premiere_semaine.text.strip() if premiere_semaine else "0"
    )

# Initialisation d'un DataFrame pour stocker les données extraites
df = pd.DataFrame(columns=['ID', 'annee', 'titre', 'budget', 'premiere_semaine'])
session = requests.Session()

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=12) as executor:
    # Soumission des tâches de scraping pour chaque ID de film entre 1 et la variable total_film
    tests = [executor.submit(jpbox, k) for k in range(id_in, id_out)]
    for test in tests:
        # Récupération des résultats et ajout au DataFrame si le titre est différent de "0"
        k, annee, titre, budget, premiere_semaine = test.result()
        if titre != "0":
            df.loc[k] = [k, annee, titre, budget, premiere_semaine]

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv(f'jpbox_{id_in}_{id_out}.csv', index=False)

# Affichage du DataFrame final
display(df)

Execution : 0.02 %
Execution : 0.22 %
Execution : 0.42 %
